In [2]:
from random import randint, uniform
from util import generate_problem, verify_witness
from matplotlib import pyplot as plt

In [3]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def sweep_line(intervals):
    L = []
    for l, r in intervals:
        L.append( (l, 1) )
        L.append( (r, -1) )
        
    L.sort(key=lambda a: 10*a[0] - a[1])
    
    c = 0
    top = -1
    top_index = 0
    
    for index, (point, increment) in enumerate(L):
        c += increment
        if c > top:
            top = c
            top_index = index
    
    sol = int( (L[top_index][0] + L[top_index+1][0])/2 )
    
    # returns the number of constraints unsatisfied and time point
    return len(intervals) - top, sol

def walk_gene(gene, T, pick_best_gene=True):
    num_variables = max([max(t['i'], t['j']) for t in T])
    
    constraints_per_variable = [list() for i in range(num_variables + 1)]
    failed_per_variable = [0 for i in range(num_variables + 1)]
    # at index i should contain availability intervals (given other variables are fixed.)
    # remark: existence of an interval is local with respect to one constraint
    # remark: so if intersection is empty, then there is no "safe" place to put this variable
    
    for c in T:
        i, j = c['i'], c['j']
        intervals = c['intervals']
        for l, r in intervals:
            # X_j - X_i in (l, r)
            # fix X_j to genes[j]
            # so genes[j] - X_i in (l, r)
            # then X_i in [X_j - r, X_j - l]
            constraints_per_variable[i].append( (gene[j] - r, gene[j] - l) )
            
            # now for j:
            constraints_per_variable[j].append( (gene[i] + l, gene[i] + r) )
            
            
            if gene[j] - gene[i] < l or gene[j] - gene[i] > r:
                failed_per_variable[i] += 1
                failed_per_variable[j] += 1
            
    
    # for each constraint, sweep line, find best option
    # (later? we want to pick whatever REDUCES the unsatisfieds the most)
    # for now: pick a random variable and modify it according to sweep_line
    # if no later, then we can make the loop upstairs simpler (to only do stuff if "i" is mentioned)

    
    if pick_best_gene:
        reduction = -1
        best_gene = -1
        new_gene_base = 0
        for i in range(1, num_variables+1):
            faileds = failed_per_variable[i]
            new_faileds, new_gene_base = sweep_line(constraints_per_variable[i])
            
            if faileds - new_faileds >= reduction: #important to have \geq so it doesnt get stuck on 1 if 1 is best
                reduction = faileds - new_faileds
                best_gene = i
                best_gene_base = new_gene_base
        gene[best_gene] = best_gene_base
    else:
        i = randint(1, num_variables)
        gene[i] = sweep_line(constraints_per_variable[i])[1]
        

    return gene
    
def walk_tcsp(T, r, max_iterations, max_flips, pick_best_gene):
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T, r)
        
        for j in range(max_flips):
            gene_failed = verify_witness(gene, T)
            
            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed
            
            if len(gene_failed) == 0:
                break
                
            gene = walk_gene(gene, T, pick_best_gene)
                
        # necessary for double break after flips loop
        if len(gene_failed) == 0: break
    
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [4]:
T = generate_problem(
    50,
    min_intervals=1,
    max_intervals=1,
    scaling_factor=1
)

In [5]:
walk_tcsp(T, 100, 50, 50, pick_best_gene=False)

num iterations: 50
best gene: [0, -41, -11, -3, -19, -15, -11, -56, 51, -25, 3, 52, -12, 15, -9, -2, -26, 7, 38, -1, 22, -7, 38, 3, 8, -36, 81, -9, -13, -36, -11, -63, 61, -38, -34, -5, 34, -8, -91, -34, 1, -50, -26, -33, 7, -18, 0, 39, -49, -53]
constraints failed: 121 out of: 336
failed constraints: [{'i': 1, 'j': 7, 'intervals': [(19, 64)]}, {'i': 1, 'j': 12, 'intervals': [(-99, -78)]}, {'i': 1, 'j': 32, 'intervals': [(-25, 47)]}, {'i': 1, 'j': 40, 'intervals': [(-39, -21)]}, {'i': 2, 'j': 12, 'intervals': [(-72, -42)]}, {'i': 2, 'j': 17, 'intervals': [(-84, -53)]}, {'i': 2, 'j': 47, 'intervals': [(-87, 23)]}, {'i': 3, 'j': 12, 'intervals': [(-65, -59)]}, {'i': 3, 'j': 21, 'intervals': [(-98, -93)]}, {'i': 3, 'j': 32, 'intervals': [(-79, 12)]}, {'i': 3, 'j': 43, 'intervals': [(19, 50)]}, {'i': 3, 'j': 45, 'intervals': [(43, 87)]}, {'i': 4, 'j': 25, 'intervals': [(-98, -79)]}, {'i': 4, 'j': 28, 'intervals': [(-43, 0)]}, {'i': 4, 'j': 34, 'intervals': [(-37, -18)]}, {'i': 4, 'j': 35, 

In [6]:
l = [(0, 10), (1, 5), (1, 6), (6, 10)]

In [7]:
sweep_line(l)

(1, 3)

In [8]:
# problem sets:
SIMPLE = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 1,
    'scaling_factor': 1,
}

BINARY = {
    'constraint_probability': 0.25,
    'min_intervals': 2,
    'max_intervals': 2,
    'scaling_factor': 1,
}

FIVE_MAX = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 5,
    'scaling_factor': 1,
}

## Graphing effectiveness of walking a gene

In [9]:
def test(T, r, max_flips):
    unsats = []
    gene = random_gene(T, r)

    for j in range(max_flips):
        gene_failed = verify_witness(gene, T)
        unsats += [len(gene_failed)]

        if len(gene_failed) == 0:
            break

        gene = walk_gene(gene, T)

    return unsats

In [10]:
runs = 50
run_results = []

for _ in range(runs):
    T = generate_problem(
        variables=20,
        **BINARY,
    )
    
    arr = test(T, r=100, max_flips=100)
    
    starting = arr[0]
    ending = arr[-1]
    decrease = (arr[0]-arr[-1]) / arr[0]
        
    # TODO, problem: we do not know if it is even solvable!
    # will have to confirm with backtracking, or make sure to generate only solvable problems
    solved = arr[-1] == 0
    
    print(starting, ending, f'{decrease:.2f}', solved)

32 4 0.88 False
24 3 0.88 False
28 3 0.89 False
28 4 0.86 False
34 8 0.76 False
38 5 0.87 False
30 5 0.83 False
31 8 0.74 False
26 6 0.77 False
28 2 0.93 False
29 7 0.76 False
29 5 0.83 False
31 5 0.84 False
28 6 0.79 False
32 6 0.81 False
25 4 0.84 False
26 7 0.73 False
25 4 0.84 False
38 6 0.84 False
27 5 0.81 False
28 4 0.86 False
34 5 0.85 False
26 6 0.77 False
23 2 0.91 False
30 8 0.73 False
28 6 0.79 False
20 6 0.70 False
19 5 0.74 False
24 5 0.79 False
23 5 0.78 False
19 5 0.74 False
26 2 0.92 False
24 4 0.83 False
22 1 0.95 False
28 4 0.86 False
30 4 0.87 False
21 4 0.81 False
27 2 0.93 False
32 3 0.91 False
32 3 0.91 False
19 4 0.79 False
26 3 0.88 False
25 3 0.88 False
25 3 0.88 False
28 8 0.71 False
27 4 0.85 False
28 4 0.86 False
31 7 0.77 False
41 4 0.90 False
35 7 0.80 False


In [11]:
# Note:
# if returned length is less, that means the heuristic solved the problem
# check whether it ever increases.

## Graphing pick_best_gene vs random gene selection for walking

In [12]:
runs = 50
run_results = []

count_same = 0
count_better = 0
count_worse = 0
reductions = []

for _ in range(runs):
    T = generate_problem(
        variables=50,
        **BINARY,
    )
    
    gene = random_gene(T, 100)
    best_gene = [i for i in gene]
    rand_gene = [i for i in gene]
    
    pick_best = len(verify_witness(
        walk_gene(best_gene, T, pick_best_gene=True),
        T)
    )
    pick_random = len(verify_witness(
        walk_gene(rand_gene, T, pick_best_gene=False),
        T)
    )
    
    reductions += [pick_random - pick_best]
    
    if pick_best == pick_random:
        count_same += 1
    elif pick_best < pick_random:
        count_better += 1
    else:
        count_worse += 1
    
print('same', count_same)
print('better', count_better)
print('worse', count_worse)

from statistics import stdev
print('avg reduction', sum(reductions)/len(reductions))
print('stddev reduction', stdev(reductions))

same 2
better 48
worse 0
avg reduction 4.64
stddev reduction 2.0279677170074697


## and now, what about repeated application of best/random?

as in, how much can it reduce, just like the random gene vs n walks test

In [13]:
runs = 5
walks = 50
run_results = []

count_same = 0
count_better = 0
count_worse = 0

for _ in range(runs):
    T = generate_problem(
        variables=10,
        **BINARY,
    )
    
    gene = random_gene(T, 100)
    best_gene = [i for i in gene]
    rand_gene = [i for i in gene]
    
    for _ in range(runs):
        best_gene = walk_gene(best_gene, T, pick_best_gene=True)
        rand_gene = walk_gene(rand_gene, T, pick_best_gene=False)
    
    pick_best = len(verify_witness(best_gene, T))
    pick_random = len(verify_witness(rand_gene, T))
    
    if pick_best == pick_random:
        count_same += 1
    elif pick_best < pick_random: # \lt because verify_witness is like golf.
        count_better += 1
    else:
        count_worse += 1
    
print('same', count_same)
print('better', count_better)
print('worse', count_worse)

same 0
better 5
worse 0
